In [ ]:
import batoid
import os
import yaml
import numpy as np
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
LSST_r_fn = os.path.join(batoid.datadir, "LSST", "LSST_r.yaml")
config = yaml.safe_load(open(LSST_r_fn))
fiducial_telescope = batoid.parse.parse_optic(config['opticalSystem'])

In [ ]:
theta_x = 0
theta_y = 0
wavelength = 620e-9

telescope = fiducial_telescope.withGloballyShiftedOptic('LSST.LSSTCamera', [0,0,1e-3])

telescope.dist = 20.0
telescope.pupil_size = 10.0
telescope.sphereRadius = 10.5
telescope.inMedium = batoid.Air()

nx=1536
fftpsf = batoid.fftPSF(telescope, theta_x, theta_y, wavelength, nx=nx, pad_factor=1)
scale = np.sqrt(np.abs(np.linalg.det(fftpsf.primitiveVectors)))  # meters

extent = scale*fftpsf.array.shape[0]/2*np.r_[-1., 1., -1., 1.] # meters
extent -= scale/2
extent *= 1e6 # microns

plt.imshow(fftpsf.array/fftpsf.array.sum(), extent=extent)
plt.colorbar()
plt.title("FFT PSF")
plt.show()

fftpsf_sub = fftpsf.array[16::32, 16::32]
plt.imshow(fftpsf_sub/fftpsf_sub.sum(), extent=extent)
plt.colorbar()
plt.title("FFT PSF downsampled")
plt.show()

In [ ]:
theta_x = 0
theta_y = np.deg2rad(1.75)
wavelength = 620e-9

telescope = fiducial_telescope.withGloballyShiftedOptic('LSST.LSSTCamera', [0,0,1e-3])

telescope.dist = 20.0
telescope.pupil_size = 10.0
telescope.sphereRadius = 10.5
telescope.inMedium = batoid.Air()

nx=1536
fftpsf = batoid.fftPSF(telescope, theta_x, theta_y, wavelength, nx=nx, pad_factor=1)
scale = np.sqrt(np.abs(np.linalg.det(fftpsf.primitiveVectors)))  # meters

extent = scale*fftpsf.array.shape[0]/2*np.r_[-1., 1., -1., 1.] # meters
extent -= scale/2
extent *= 1e6 # microns

plt.imshow(fftpsf.array/fftpsf.array.sum(), extent=extent)
plt.colorbar()
plt.title("FFT PSF")
plt.show()

fftpsf_sub = fftpsf.array[16::32, 16::32]
plt.imshow(fftpsf_sub/fftpsf_sub.sum(), extent=extent)
plt.colorbar()
plt.title("FFT PSF downsampled")
plt.show()

In [ ]:
wavelength = 500e-9
theta_x = 0
theta_y = np.deg2rad(1.75)

dirCos = batoid.utils.gnomicToDirCos(theta_x, theta_y)

telescope = fiducial_telescope.withGloballyShiftedOptic('LSST.LSSTCamera', [0,0,1e-3])

telescope.dist = 20.0
telescope.pupil_size = 8.36
telescope.sphereRadius = 10.5
telescope.inMedium = batoid.Air()

nx = 1800
nxOut = 32
dx = 1200*1e-6/nxOut
huygensPSF = batoid.huygensPSF(telescope, theta_x, theta_y, wavelength, nx=nx, dx=dx, dy=dx, nxOut=nxOut)

scale = np.sqrt(np.abs(np.linalg.det(huygensPSF.primitiveVectors)))  # meters
if huygensPSF.primitiveVectors[0,0] < 1:
    huygensPSF.array = huygensPSF.array[::-1,::-1]

extent = scale*huygensPSF.array.shape[0]/2*np.r_[-1., 1., -1., 1.]  # meters
extent -= scale/2
extent *= 1e6 # microns

plt.imshow(huygensPSF.array/huygensPSF.array.sum(), extent=extent)
plt.colorbar()
plt.title("Huygens PSF")
plt.show()